In [25]:
import numpy as np
from keras.layers import Dropout
from sklearn.datasets import load_files    
from keras.utils import np_utils
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint  
from keras.optimizers import SGD
import tensorflow as tf
import keras
import cv2
import os
import pandas as pd
import pdb
from keras.preprocessing.image import ImageDataGenerator

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [26]:
transferModel = "xception"
trainDir = 'dogImages/train/'
valdDir = 'dogImages/valid/'
testDir = 'dogImages/test/'
img_width = img_height = 224
batch_size = 16

In [27]:
def renameDirs(path):
    dirs = os.listdir(path)
    basePath = os.getcwd()
    for ddir in dirs:
        if '.' in ddir:
            newdir = ddir.split('.')[1]
            os.rename(os.path.join(basePath,path,ddir), os.path.join(basePath,path,newdir)) 

In [28]:
renameDirs(trainDir)
renameDirs(valdDir)
renameDirs(testDir)

In [29]:
def get_nbSmaples(path):
    noOfFiles = 0
    for base, dirs, files in os.walk(path):
        for Files in files:
            noOfFiles += 1
    return(noOfFiles)

In [30]:
def computeBottleneckFeat(imagePath):
    model = keras.applications.Xception(weights='imagenet',include_top=False)
    im = cv2.resize(cv2.imread(imagePath), (img_width, img_height)).astype(np.float32)
    im = np.expand_dims(im, axis=0)
    im_transform = model.predict(im*1./255)
    print('Features Generated: ',im_transform.shape)
    return(im_transform)

In [31]:
def load_dataset(path,targ_names = False):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 133)
    
    if targ_names:
        return dog_files, dog_targets,data['target_names']
    else:
        return dog_files, dog_targets

In [32]:
def predict(model, imagePath):
    
    feats = computeBottleneckFeat(imagePath)
    predict_proba = model.predict(feats)
    pred = predict_proba.argmax(axis=-1)[0]
    pred_label = train_target_names[pred]
    
    img = cv2.imread(testImage)
    img = cv2.putText(img, pred_label, (10,50), cv2.FONT_HERSHEY_SIMPLEX ,  
                       1, (255,0,0), 2, cv2.LINE_AA)
    cv2.imshow(pred_label,img)
    cv2.waitKey(0) # waits until a key is pressed
    cv2.destroyAllWindows() # destroys the window showing image

    
    return(pred_label)

In [33]:
def createModel(inputShape):
    
    # Model Creation
    Xception_model = Sequential()
    Xception_model.add(GlobalAveragePooling2D(input_shape=inputShape))
    Xception_model.add(Dense(500, activation='relu'))
    Xception_model.add(Dropout(0.4))
    Xception_model.add(Dense(133, activation='softmax'))
    Xception_model.summary()
    
    # Compile
    Xception_model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.002), metrics=['accuracy'])
    return(Xception_model)

In [34]:
def trainModel(model):
    checkpointer = ModelCheckpoint(filepath='models/weights.best.Xception.hdf5', 
                               verbose=1, save_best_only=True)
    model.fit(train_Xception, train_targets, 
          validation_data=(valid_Xception, valid_targets),
          epochs=120, batch_size=20, callbacks=[checkpointer], verbose=1)
    return(model)

In [35]:
def loadModel(inputShape,weights):
    Xception_model = Sequential()
    Xception_model.add(GlobalAveragePooling2D(input_shape=inputShape))
    Xception_model.add(Dense(500, activation='relu'))
    Xception_model.add(Dropout(0.4))
    Xception_model.add(Dense(133, activation='softmax'))
    Xception_model.load_weights(weights)
    return(Xception_model)

In [36]:
def testModel(model,test_Xception,test_targets,test_target_names):
    Xception_predictions = [np.argmax(model.predict(np.expand_dims(feature, axis=0))) for feature in test_Xception]

    # evaluating on test data
    test_accuracy = 100*np.sum(np.array(Xception_predictions)==np.argmax(test_targets, axis=1))/len(Xception_predictions)
    print('Test accuracy: %.4f%%' % test_accuracy)
    targs = np.array(test_target_names)
    preds = list(targs[Xception_predictions])
    
    actual = np.argmax(test_targets,axis = 1)
    actual = np.array(actual)
    actual = list(targs[actual])
    
    print(classification_report(actual, preds, target_names=test_target_names))
    
    

In [41]:
train_files, train_targets, train_target_names = load_dataset('dogImages/train',True)
valid_files, valid_targets, valid_target_names= load_dataset('dogImages/valid',True)
test_files, test_targets, test_target_names= load_dataset('dogImages/test',True)

In [42]:
bottleneck_features = np.load('bottleneck_feat/DogXceptionData.npz')
train_Xception = bottleneck_features['train']
valid_Xception = bottleneck_features['valid']
test_Xception = bottleneck_features['test']
train_Xception.shape,test_Xception.shape,valid_Xception.shape

((6680, 7, 7, 2048), (836, 7, 7, 2048), (835, 7, 7, 2048))

In [43]:
xceptionModel = createModel(train_Xception.shape[1:])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               1024500   
_________________________________________________________________
dropout_4 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 133)               66633     
Total params: 1,091,133
Trainable params: 1,091,133
Non-trainable params: 0
_________________________________________________________________


In [19]:
xceptionModel = loadModel(train_Xception.shape[1:],'models/weights.best.Xception.hdf5')
xceptionModel.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               1024500   
_________________________________________________________________
dropout_1 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 133)               66633     
Total params: 1,091,133
Trainable params: 1,091,133
Non-trainable params: 0
_________________________________________________________________


In [44]:
xceptionModel = trainModel(xceptionModel)

Train on 6680 samples, validate on 835 samples
Epoch 1/120
6680/6680 [==============================] - 4s 542us/step - loss: 4.7961 - accuracy: 0.0323 - val_loss: 4.5415 - val_accuracy: 0.1269

Epoch 00001: val_loss improved from inf to 4.54151, saving model to models/weights.best.Xception.hdf5
Epoch 2/120
6680/6680 [==============================] - 3s 493us/step - loss: 4.3862 - accuracy: 0.1401 - val_loss: 4.1457 - val_accuracy: 0.3449

Epoch 00002: val_loss improved from 4.54151 to 4.14571, saving model to models/weights.best.Xception.hdf5
Epoch 3/120
6680/6680 [==============================] - 3s 493us/step - loss: 3.9723 - accuracy: 0.3058 - val_loss: 3.7109 - val_accuracy: 0.4946

Epoch 00003: val_loss improved from 4.14571 to 3.71089, saving model to models/weights.best.Xception.hdf5
Epoch 4/120
6680/6680 [==============================] - 3s 492us/step - loss: 3.5283 - accuracy: 0.4290 - val_loss: 3.2466 - val_accuracy: 0.5832

Epoch 00004: val_loss improved from 3.71089 to 

6680/6680 [==============================] - 3s 498us/step - loss: 0.5975 - accuracy: 0.8427 - val_loss: 0.5769 - val_accuracy: 0.8371

Epoch 00033: val_loss improved from 0.58409 to 0.57690, saving model to models/weights.best.Xception.hdf5
Epoch 34/120
6680/6680 [==============================] - 3s 507us/step - loss: 0.5847 - accuracy: 0.8407 - val_loss: 0.5709 - val_accuracy: 0.8419

Epoch 00034: val_loss improved from 0.57690 to 0.57093, saving model to models/weights.best.Xception.hdf5
Epoch 35/120
6680/6680 [==============================] - 3s 513us/step - loss: 0.5823 - accuracy: 0.8449 - val_loss: 0.5658 - val_accuracy: 0.8407

Epoch 00035: val_loss improved from 0.57093 to 0.56579, saving model to models/weights.best.Xception.hdf5
Epoch 36/120
6680/6680 [==============================] - 3s 514us/step - loss: 0.5698 - accuracy: 0.8449 - val_loss: 0.5607 - val_accuracy: 0.8431

Epoch 00036: val_loss improved from 0.56579 to 0.56072, saving model to models/weights.best.Xceptio


Epoch 00065: val_loss improved from 0.48065 to 0.47907, saving model to models/weights.best.Xception.hdf5
Epoch 66/120
6680/6680 [==============================] - 4s 529us/step - loss: 0.4031 - accuracy: 0.8847 - val_loss: 0.4790 - val_accuracy: 0.8491

Epoch 00066: val_loss improved from 0.47907 to 0.47902, saving model to models/weights.best.Xception.hdf5
Epoch 67/120
6680/6680 [==============================] - 3s 497us/step - loss: 0.4037 - accuracy: 0.8841 - val_loss: 0.4790 - val_accuracy: 0.8491

Epoch 00067: val_loss improved from 0.47902 to 0.47897, saving model to models/weights.best.Xception.hdf5
Epoch 68/120
6680/6680 [==============================] - 3s 515us/step - loss: 0.3928 - accuracy: 0.8909 - val_loss: 0.4752 - val_accuracy: 0.8491

Epoch 00068: val_loss improved from 0.47897 to 0.47519, saving model to models/weights.best.Xception.hdf5
Epoch 69/120
6680/6680 [==============================] - 3s 519us/step - loss: 0.3983 - accuracy: 0.8865 - val_loss: 0.4750 - v

6680/6680 [==============================] - 3s 503us/step - loss: 0.3184 - accuracy: 0.9091 - val_loss: 0.4503 - val_accuracy: 0.8539

Epoch 00099: val_loss improved from 0.45057 to 0.45033, saving model to models/weights.best.Xception.hdf5
Epoch 100/120
6680/6680 [==============================] - 3s 517us/step - loss: 0.3205 - accuracy: 0.9064 - val_loss: 0.4487 - val_accuracy: 0.8539

Epoch 00100: val_loss improved from 0.45033 to 0.44874, saving model to models/weights.best.Xception.hdf5
Epoch 101/120
6680/6680 [==============================] - 3s 523us/step - loss: 0.3194 - accuracy: 0.9072 - val_loss: 0.4497 - val_accuracy: 0.8551

Epoch 00101: val_loss did not improve from 0.44874
Epoch 102/120
6680/6680 [==============================] - 4s 531us/step - loss: 0.3136 - accuracy: 0.9112 - val_loss: 0.4495 - val_accuracy: 0.8539

Epoch 00102: val_loss did not improve from 0.44874
Epoch 103/120
6680/6680 [==============================] - 3s 501us/step - loss: 0.3079 - accuracy: 

In [20]:
testModel(xceptionModel,test_Xception,test_targets,test_target_names)

Test accuracy: 84.8086%
                                    precision    recall  f1-score   support

                     Affenpinscher       0.88      0.88      0.88         8
                      Afghan_hound       0.88      0.88      0.88         8
                  Airedale_terrier       1.00      1.00      1.00         6
                             Akita       1.00      0.88      0.93         8
                  Alaskan_malamute       0.91      1.00      0.95        10
               American_eskimo_dog       1.00      1.00      1.00         8
                 American_foxhound       1.00      0.86      0.92         7
    American_staffordshire_terrier       1.00      1.00      1.00         8
            American_water_spaniel       1.00      0.50      0.67         4
            Anatolian_shepherd_dog       0.86      1.00      0.92         6
             Australian_cattle_dog       1.00      1.00      1.00         9
               Australian_shepherd       1.00      0.89      0.

In [22]:
testImage = 'dogImages/test/Akita/Akita_00258.jpg'
label = predict(xceptionModel,testImage)
print('Predicted breed is :',label)

Features Generated:  (1, 7, 7, 2048)
Predicted breed is : Akita
